# Envision 2022

## Basic Setup

Importing and Loading the URDF/SDF

In [2]:
import pybullet as p

physicsClientID = p.connect(p.GUI)

import pybullet_data

p.setAdditionalSearchPath(pybullet_data.getDataPath())

pybullet build time: Oct 11 2021 21:00:24


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce MX230/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 470.129.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 470.129.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce MX230/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [4]:
def loadFiles():
    plane = p.loadURDF('plane.urdf')

    print(plane)

    robot_StartPos = [0,0,0]
    robot_StartOrientation = p.getQuaternionFromEuler([0,0,0])

    robot = p.loadURDF("kuka_experimental/kuka_lbr_iiwa_support/urdf/lbr_iiwa_14_r820.urdf",robot_StartPos, robot_StartOrientation)

    numOfJoints = p.getNumJoints(robot)

    print(f"Plane = {plane} \nStart Position = {robot_StartPos} \nStart Orientation = {robot_StartOrientation} \nRobot = {robot} \nNumber of joints = {numOfJoints}")

    # print("Joint Details :")
    # for i in range(numOfJoints):
    #     print(p.getJointInfo(robot, i))
        
    return plane, robot, robot_StartPos, robot_StartOrientation

In [3]:
# trying to load sdf file

def load_sdf():

    # foam_brick = p.loadSDF("drake/examples/manipulation_station/models/061_foam_brick.sdf")
    # amazon_table = p.loadSDF("drake/examples/manipulation_station/models/amazon_table_simplified.sdf")
    bin1 = p.loadSDF("drake/examples/manipulation_station/models/bin.sdf")
    bin2 = p.loadSDF("drake/examples/manipulation_station/models/bin.sdf")
    gripper = p.loadSDF("drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50.sdf")
    # gripper = p.loadSDF("drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf")
    # cupboard = p.loadSDF("drake/examples/manipulation_station/models/cupboard.sdf")
    # cylinder = p.loadSDF("drake/examples/manipulation_station/models/cylinder.sdf")
    # sphere = p.loadSDF("drake/examples/manipulation_station/models/sphere.sdf")
    # thin_box = p.loadSDF("drake/examples/manipulation_station/models/thin_box.sdf")
    # thin_cylinder = p.loadSDF("drake/examples/manipulation_station/models/thin_cylinder.sdf")

    print("loaded!")
    # return [foam_brick,amazon_table,bin,cupboard,cylinder,sphere,thin_box,thin_cylinder]
    return bin1, bin2, gripper

In [56]:
bin1, bin2, gripper = load_sdf()
bin1_position = [1,0,0]
bin1_Orientation = p.getQuaternionFromEuler([0,0,0])
p.resetBasePositionAndOrientation(bin1[0],bin1_position,bin1_Orientation)

bin2_position = [0,1,0]
bin2_Orientation = p.getQuaternionFromEuler([0,0,1.57079632679])
p.resetBasePositionAndOrientation(bin2[0],bin2_position,bin2_Orientation)

gripper_position = [1,1,0]
gripper_Orientation = p.getQuaternionFromEuler([0,0,0])
p.resetBasePositionAndOrientation(gripper[0],gripper_position,gripper_Orientation)



b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: expected a world element, continuing without it.
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: expected a world element, continuing without it.
loaded!
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: expected a world element, continuing without it.


In [45]:
p.setRealTimeSimulation(0)
p.resetSimulation()

## Setting Up Camera

In [6]:
def getImage(robot) :
    
    # The Camera will be placed on the Link 7 and will always be looking downwards
    
    # The Position and orientation of the Link 7 will be used to determine the camera position and orientation
    cameraPosition = p.getLinkState(robot, 7)[0]
    cameraOrientation = p.getEulerFromQuaternion(p.getLinkState(robot, 7)[1])

    print(f"\nCamera Position: {cameraPosition} \nCamera Orientation: {cameraOrientation}")

    # The viewMatrix The camera view matrix is a complicated 4x4 matrix, but in simplest terms it describes where the camera is and in what direction it is pointing. There are some useful helper functions for creating this matrix by specifying position and rotation more directly. The function computeViewMatrix can create this matrix in exchange for three vectors.
    viewMatrix = p.computeViewMatrix(
        # cameraEyePosition describes the physical location of the camera in x, y, and z coordinates
        cameraEyePosition = cameraPosition,
        # cameraTargetPosition describes the point that we wish the camera to face : Negative Z direction, that is downwards
        cameraTargetPosition = [0, 0, -1],
        # cameraUpVector describes the orientation of the camera
        cameraUpVector = cameraOrientation)

    # The projection matrix, much like the view matrix, can be created using a few helper functions. In this case, the computeProjectionMatrixFOV function describes our camera’s intrinsic properties in the simplest and most pertinent ways to our use case.
    projectionMatrix = p.computeProjectionMatrixFOV(
        # Field of View
        fov = 45.0,
        # Aspect ratio
        aspect = 1.0,
        # The minimum distance the camera will render objects
        nearVal = 0.1,
        # The maximum distance the camera will render objects
        farVal = 10.1)

    # This function returns three image buffers: rgbImg, depthImg, and segImg. rgbImg is a uint8 image with red, green, blue, and alpha channels of the camera’s visuals. depthImg is a floating-point grayscale image that describes the distance of individual rendered pixels from the camera. It can be used to model the field-of-view of a real-world depth sensor. Lastly, segImg is a segmentation mask of the image where pixels each contain unique integers with object IDs. These are invaluable for training segmentation algorithms for robotic agents, such as a robotic arm that needs to identify objects to sort into respective bins or for a driverless car that wants to identify pedestrians, street signs, and roads.
    width, height, rgbImg, depthImg, segImg = p.getCameraImage(
        # Width of the image in pixels
        width = 224, 
        # Height of the image in pixels
        height = 224,
        # view matrix as the one deifined above
        viewMatrix = viewMatrix,
        # projectionMatrix as the one deifined above
        projectionMatrix = projectionMatrix)
    
    return width, height, rgbImg, depthImg, segImg

## Function Calls

In [57]:
plane, robot, robot_StartPos, robot_StartOrientation = loadFiles()

3
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: link_1
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: link_2
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial d

In [48]:
# reset pos and orientation of robot
# p.resetBaseVelocity(robot)
p.resetBasePositionAndOrientation(robot,robot_StartPos,robot_StartOrientation)

In [58]:
# fix base of robot to plane

cid = p.createConstraint(robot, -1, plane, -1, p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], [0, 0, 0])
print(cid)
print(p.getConstraintUniqueId(0))

# fix bins to plane

cid = p.createConstraint(bin1[0], -1, plane, -1, p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], bin1_position, childFrameOrientation = bin1_Orientation)
print(cid)
print(p.getConstraintUniqueId(0))

cid = p.createConstraint(bin2[0], -1, plane, -1, p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], bin2_position, childFrameOrientation = bin2_Orientation)
print(cid)
print(p.getConstraintUniqueId(0))

# cid = p.createConstraint(gripper[0], -1, plane, -1, p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], gripper_position, childFrameOrientation = gripper_Orientation)
# print(cid)
# print(p.getConstraintUniqueId(0))


1
1
2
1
3
1


In [51]:
_link_name_to_index = {p.getBodyInfo(gripper[0])[0].decode('UTF-8'):-1,}
        
for _id in range(p.getNumJoints(gripper[0])):
	_name = p.getJointInfo(gripper[0], _id)[12].decode('UTF-8')
	_link_name_to_index[_name] = _id

In [24]:
print(_link_name_to_index)

{'body': -1, 'left_finger': 0, 'right_finger': 1}


In [52]:
_link_name_to_index = {p.getBodyInfo(robot)[0].decode('UTF-8'):-1,}
        
for _id in range(p.getNumJoints(robot)):
	_name = p.getJointInfo(robot, _id)[12].decode('UTF-8')
	_link_name_to_index[_name] = _id

In [53]:
print(_link_name_to_index)

{'base_link': -1, 'link_1': 0, 'link_2': 1, 'link_3': 2, 'link_4': 3, 'link_5': 4, 'link_6': 5, 'link_7': 6, 'tool0': 7, 'base': 8}


In [59]:
# contraint the body and gripper
cid_g = p.createConstraint(robot, 6, gripper[0], -1, p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], gripper_position, childFrameOrientation = gripper_Orientation)
print(cid_g)
print(p.getConstraintUniqueId(0))


4
1


In [ ]:
# realtime camera analysis
import time
for _ in range(10000):
    width, height, rgbImg, depthImg, segImg = getImage(robot)
    time.sleep(0.1)

## Simulation parameters

In [ ]:
p.setGravity(0, 0, 0)

In [34]:
p.setGravity(0, 0, -9.81)

In [41]:
p.setRealTimeSimulation(1)

In [29]:
p.setRealTimeSimulation(0)

In [55]:
p.setRealTimeSimulation(0)
p.resetSimulation()

## Disconnect from the Server

In [ ]:
p.disconnect()

## Rough

In [ ]:
print(p.getJointInfo(robot, 6))
print("Position = ", p.getJointInfo(robot, 6)[14])
print("Orientation", p.getEulerFromQuaternion(p.getJointInfo(robot, 6)[15]))

In [ ]:
p.setJointMotorControl2(robot, 5, p.VELOCITY_CONTROL, targetVelocity =  -0.2)

In [ ]:
p.setJointMotorControlArray(robot, range(9), p.POSITION_CONTROL, targetPositions =  [0]*9)

In [ ]:
p.setJointMotorControl2(robot, 1, p.POSITION_CONTROL, targetPosition =  0.5)